### Bibliotecas

In [2]:
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

### Iniciando uma instância no Google Chrome

In [14]:
# A classe Service é usada para iniciar uma instância do Chrome WebDriver
service = Service()

# webdriver.ChromeOptions é usada para definir a preferência para o browser do Chrome
options = webdriver.ChromeOptions()

# Inicia-se a instância do Chrome WebDriver com as definidas 'options' e 'service'
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.gov.br/pf/pt-br/search?origem=form&SearchableText=licitacao"

driver.get(url)

### Encontrar Elementos do HTML
- find_element(By.ID, 'id')
- find_element(By.NAME, 'name')
- find_element(By.XPATH, 'xpath')
- find_element(By.LINK_TEXT, 'link text')
- find_element(By.PARTIAL_LINK_TEXT, 'partial link text')
- find_element(By.TAG_NAME, 'tag name')
- find_element(By.CLASS_NAME, 'class name')
- find_element(By.CSS_SELECTOR, 'css selector')

In [31]:
elementos = driver.find_elements(By.CSS_SELECTOR, 'span.titulo a')
print(elementos)

[<selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.454")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.455")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.456")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.457")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9cb8b3d8b", element="f.A678B909822C47EA2DA281E8EA6C8583.d.87DB4FDA214C8E9C803CCE242BD30409.e.458")>, <selenium.webdriver.remote.webelement.WebElement (session="25beacadc9b81df022b9b2d9c

In [33]:
len(elementos)

30

In [32]:
for elemento in elementos:
    print('Texto da notícia: ', elemento.text)
    print("Link: ", elemento.get_attribute('href'))
    print('##################################')

Texto da notícia:  PF e CGU deflagram operação de combate a crimes contra à administração pública e de lavagem de capitais em Pernambuco
Link:  https://www.gov.br/pf/pt-br/assuntos/noticias/2025/06/pf-e-cgu-deflagram-operacao-de-combate-a-crimes-contra-a-administracao-publica-e-de-lavagem-de-capitais-em-pernambuco
##################################
Texto da notícia:  PF desarticula esquema criminoso de desvio de verbas da saúde e da educação no interior do Piauí
Link:  https://www.gov.br/pf/pt-br/assuntos/noticias/2025/06/pf-desarticula-esquema-criminoso-de-desvio-de-verbas-da-saude-e-da-educacao-no-interior-do-piaui
##################################
Texto da notícia:  PF e PMSC prendem uruguaio procurado pela Interpol e apreendem droga, veículos e dinheiro em espécie
Link:  https://www.gov.br/pf/pt-br/assuntos/noticias/2025/05/pf-e-pmsc-prendem-cidadao-uruguaio-procurado-pela-interpol-e-apreende-droga-veiculos-e-dinheiro-em-especie
##################################
Texto da notícia:

In [ ]:
# //*[@id="search-results"]/div[4]/div[2]/span[2]/ul[2]/li[4]/a[1]

In [67]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import pandas as pd
from openai import OpenAI
from tqdm import tqdm
import time
from dotenv import load_dotenv
import os

In [21]:

def iniciar_driver():
    service = Service()
    # webdriver.ChromeOptions é usada para definir a preferência para o browser do Chrome
    options = webdriver.ChromeOptions()

    # Inicia-se a instância do Chrome WebDriver com as definidas 'options' e 'service'
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [23]:

termo_busca = 'licitacao'

def coletar_todas_paginas():
    driver = iniciar_driver()
    url = f"https://www.gov.br/pf/pt-br/search?SearchableText={termo_busca}"
    driver.get(url)
    time.sleep(2)

    todas_noticias = []
    pagina = 1

    while True:
        print(f"\n🔎 Coletando dados da página {pagina}")

        # Pega os títulos
        noticias = driver.find_elements(By.CSS_SELECTOR, "span.titulo a")
        for noticia in noticias:
            titulo = noticia.text.strip()
            link = noticia.get_attribute("href")
            todas_noticias.append({"Título": titulo, "Link": link})
            print(f"✔ {titulo}")

        # Tenta localizar o botão "Próximo »"
        try:
            proximo = driver.find_element(By.XPATH, "//a[@class='proximo' and contains(text(), 'Próximo')]")
            time.sleep(1)
            driver.execute_script("arguments[0].scrollIntoView();", proximo)
            time.sleep(1)

            # Força o clique via JavaScript
            driver.execute_script("arguments[0].click();", proximo)

            pagina += 1
            time.sleep(2)
        except NoSuchElementException:
                print("🚫 Fim da paginação.")
                break

    driver.quit()
    return todas_noticias



In [ ]:
dados = coletar_todas_paginas()

In [ ]:

pd.DataFrame(dados).to_csv("noticias_licitacao_pf.csv", index=False, encoding="utf-8")
print(f"\n✅ Total de notícias coletadas: {len(dados)}")


✅ Total de notícias coletadas: 830


### Extração dos textos dentro da notícia

In [24]:
caminho_arquivo = 'noticias_licitacao_pf.csv'

dados_arq = pd.read_csv(caminho_arquivo, )

print(dados_arq['Link'])


0      https://www.gov.br/pf/pt-br/assuntos/noticias/...
1      https://www.gov.br/pf/pt-br/assuntos/noticias/...
2      https://www.gov.br/pf/pt-br/assuntos/noticias/...
3      https://www.gov.br/pf/pt-br/assuntos/noticias/...
4      https://www.gov.br/pf/pt-br/assuntos/noticias/...
                             ...                        
825    https://www.gov.br/pf/pt-br/assuntos/noticias/...
826    https://www.gov.br/pf/pt-br/assuntos/noticias/...
827    https://www.gov.br/pf/pt-br/assuntos/noticias/...
828    https://www.gov.br/pf/pt-br/assuntos/noticias/...
829    https://www.gov.br/pf/pt-br/assuntos/noticias/...
Name: Link, Length: 830, dtype: object


In [ ]:
def extrair_texto_principal_noticia():
    driver = iniciar_driver()
    dados_noticia = []

    for i in range(0 , len(dados_arq['Link'])):
        print(f"✅ Coletando dados da notícia {i + 1} de {len(dados_arq['Link'])}...")
        url = dados_arq['Link'][i]
        driver.get(url)
        time.sleep(0.3)
        data_publicacao = driver.find_element(By.CLASS_NAME, "value").text.strip()
        titulo = driver.find_element(By.CLASS_NAME, 'documentFirstHeading').text.strip()
        descricao = driver.find_element(By.CLASS_NAME, "documentDescription").text.strip()
        texto_principal = driver.find_element(By.ID, "content-core").text.strip()
        dados_noticia.append({
            "Data publicação:": data_publicacao,
            "Título": titulo ,
            "Descrição": descricao ,
            "Texto Principal": texto_principal,
            'URL': url
            })

    driver.quit()
    return dados_noticia

In [57]:
resultado = extrair_texto_principal_noticia()

pd.DataFrame(resultado).to_csv("dados_noticias.csv", index=False, encoding="utf-8")
print(f"\n✅ Total de notícias coletadas: {len(resultado)}")

✅ Coletando dados da notícia 1 de 830...
✅ Coletando dados da notícia 2 de 830...
✅ Coletando dados da notícia 3 de 830...
✅ Coletando dados da notícia 4 de 830...
✅ Coletando dados da notícia 5 de 830...
✅ Coletando dados da notícia 6 de 830...
✅ Coletando dados da notícia 7 de 830...
✅ Coletando dados da notícia 8 de 830...
✅ Coletando dados da notícia 9 de 830...
✅ Coletando dados da notícia 10 de 830...
✅ Coletando dados da notícia 11 de 830...
✅ Coletando dados da notícia 12 de 830...
✅ Coletando dados da notícia 13 de 830...
✅ Coletando dados da notícia 14 de 830...
✅ Coletando dados da notícia 15 de 830...
✅ Coletando dados da notícia 16 de 830...
✅ Coletando dados da notícia 17 de 830...
✅ Coletando dados da notícia 18 de 830...
✅ Coletando dados da notícia 19 de 830...
✅ Coletando dados da notícia 20 de 830...
✅ Coletando dados da notícia 21 de 830...
✅ Coletando dados da notícia 22 de 830...
✅ Coletando dados da notícia 23 de 830...
✅ Coletando dados da notícia 24 de 830...
✅

In [68]:
#Carregando a chave de API
load_dotenv()
api_key = os.getenv('OPEN_API_KEY')
client =OpenAI(api_key=api_key)

In [ ]:
# FUNÇÃO PARA CLASSIFICAR A NOTÍCIA
def classificar_noticia(titulo, descricao, texto):
    prompt = f"""
Você é um especialista em classificar notícias. Você receberá várias notícias e cada uma terá um título, uma descrição e um texto principal.
A grande maioria das notícias são sobre fraudes em licitações publicadas no site da polícia federal brasileira.
Sua função classificar a notícia tendo como base campos: título, descrição e texto principal. A classificação deve retornar somente os termos abaixo.

- obra - para fraudes em licitações na área da construção civil, como prédios, pontes ou outras.
- saúde - para fraudes em licitações envolvendo a área da saúde como compras de medicamentos superfaturados, compra de equipamentos médicos e outros.
- escola - para fraude envolvendo o setor de educação como superfaturamento na compra de merendas, ou outras áreas do setor de educação.
- serviços de terceiros - para fraudes como contração de empresas com superfaturamento ou que não entregam os serviços dentre outras.
- TI - para fraude na área de tecnologia, como compra de computadores, equipamentos de informática em geral ou prestação de serviço na área de TI e outras.
- outras - para fraude de licitações que não se encaixam nas áreas descritas anteriormente.
- não fraude - para notícias que não são fraudes em licitações, são outras investigações ou atuação da polícia federal que não se encaixa em uma fraude de licitação.

Importante levar em consideração que nem todas as notícias serão de fraude em licitação. Outro ponto importante é que algumas fraudes em licitação podem ser classificadas
por mais de um dos termos descritos anteriormente. Por exemplo, a fraude de licitação se deu na construção de uma hospital, isso envolve o termo obra e saúde, nestes
casos classifique a fraude com os dois ou mais termos sugeridos.

Título: {titulo}
Descrição: {descricao}
Texto: {texto}

Retorne **apenas** o nome da categoria apropriada.
"""
    try:
        resposta = client.chat.completions.create(
            model="gpt-4o",  # ou "gpt-3.5-turbo"
            messages=[
                {"role": "system", "content": "Você é um especialista muito prestativo e cuidadoso que classifica notícias sobre fraudes em licitações e outros tipos de crimes."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,
        )
        categoria = resposta.choices[0].message.content.strip().lower()
        return categoria
    except Exception as e:
        print(f"Erro na API: {e}")
        return "erro"

In [76]:
# CARREGAR O CSV
df = pd.read_csv("dados_noticias.csv")

In [77]:
df.head(5)

,Data publicação:,Título,Descrição,Texto Principal,URL
0,05/06/2025 09h22,PF e CGU deflagram operação de combate a crime...,Operação objetiva reprimir a atuação de uma or...,"Recife/PE. A Polícia Federal em Pernambuco, co...",https://www.gov.br/pf/pt-br/assuntos/noticias/...
1,03/06/2025 07h36,PF desarticula esquema criminoso de desvio de ...,Três mandados de busca e apreensão são cumprid...,"Teresina/PI. A Polícia Federal deflagrou, nest...",https://www.gov.br/pf/pt-br/assuntos/noticias/...
2,30/05/2025 11h43,PF e PMSC prendem uruguaio procurado pela Inte...,O foragido possuía mandado de prisão preventiv...,"Florianópolis/SC. A Polícia Federal, em ação c...",https://www.gov.br/pf/pt-br/assuntos/noticias/...
3,21/05/2025 10h33,PF deflagra operação contra fraudes em licitaç...,Foram cumpridos dois mandados de busca e apree...,"Manaus/AM. A Polícia Federal cumpriu, na manhã...",https://www.gov.br/pf/pt-br/assuntos/noticias/...
4,21/05/2025 08h18,"PF, CGU e Receita Federal deflagram operação c...",Ação conjunta com a CGU e a Receita Federal ap...,"Campo Grande/MS. A Polícia Federal, em ação co...",https://www.gov.br/pf/pt-br/assuntos/noticias/...


In [78]:

# CLASSIFICAR CADA LINHA
categorias = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    categoria = classificar_noticia(row["Título"], row["Descrição"], row["Texto Principal"])
    categorias.append(categoria)
    time.sleep(1)  # Evita atingir o limite de taxa da API

# ADICIONAR CATEGORIA AO DATAFRAME
df["categoria"] = categorias

# SALVAR RESULTADO
df.to_csv("noticias_classificadas.csv", index=False)

print("Classificação concluída. Arquivo salvo como 'noticias_classificadas.csv'.")


100%|██████████| 830/830 [44:15<00:00,  3.20s/it]  

Classificação concluída. Arquivo salvo como 'noticias_classificadas.csv'.
